In [ ]:
import pull_optionm_api_data as pull_optionm
import clean_bloomberg as clean_bbg
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from settings import config


$$
1 + f_{t}^{\pi, \tau_2}
= \frac{1 + r_{t}^{f}\,\tau_2}{1 + r_{t}^{f}\,\tau_1}
= \frac{F_{t,\tau_2} + \mathbb{E}_{t}^{Q}[D_{t,\tau_2}]}{F_{t,\tau_1} + \mathbb{E}_{t}^{Q}[D_{t,\tau_1}]}
$$

As can be seen above, we need futures close data for the two closest maturity and the expected dividend yield as well.
Expected dividend yield can be approximated with implied dividend yield as this is the market expectation embedded in index option.

STEP1 Implied Dividend yield from index options data 

In [ ]:
spx_div = pull_optionm.pull_index_implied_dividend_yield('SPX')

In [ ]:
spx_div.head()

Since we only need index dividend yield for nearest and deferred month contract, filter only those expiration dates

In [ ]:
spx_div_filtered= pull_optionm.filter_index_implied_dividend_yield(spx_div)
spx_div_filtered.head()

We pull futures price data from the bloomberg thus, we need to clean this data as well

Note that start date is before our scope of analysis.
The reason for this is because when given a target date it looks for in which range of expiration is in between and then returns the one with nearest expiry and the deferred contract. But this logic can be improved.
Moreover, even though ES1, ES2 is the first two contracts with closest expiry the reason why need ES3 is that on expiration date, we need to roll over to the next contract

STEP2 Futures close data for each index

In [ ]:
df_raw = pd.read_parquet('../data_manual/bloomberg_historical_data.parquet') # can be automated as well

start_date = datetime.strftime(config("START_DATE").date()-relativedelta(years=1),format="%Y-%m-%d") 
end_date = datetime.strftime(config("END_DATE"),format="%Y-%m-%d")

expiration_months = [3, 6, 9, 12]
expiration_dates = pull_optionm.get_expiration_dates(start_date, end_date, expiration_months)

# near future expiration date and far future expiration date for each dates
date_ranges = [(start.date(), end.date()) for start, end in zip(expiration_dates, expiration_dates[1:])]

spx = ['ES1 Index','ES2 Index','ES3 Index']
spx_pairs= list(zip(spx,spx[1:]))

index_future_spx = clean_bbg.get_clean_df (df_raw, date_ranges, spx_pairs)
index_future_spx.head()

STEP3 Convert dividend yield to a index level

See below again at column rate, we need to multiply index level to convert the rate into a index level.
Also preferably, it is easier to calculate further stuffs if we make each of the expiration have different columns

In [ ]:
spx_div_filtered.head()

In [ ]:
index_pairs_map = {
    "SPX": spx_pairs
}

index_optionm_df= dict()
index_future_df= dict()
index_df= dict()

In [ ]:
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

In [ ]:
df_div_yield = pull_optionm.pull_index_implied_dividend_yield(
        "SPX", start_date=START_DATE, end_date=END_DATE
    )

df_loaded = pull_optionm.load_index_implied_dividend_yield("SPX")

df_filtered = pull_optionm.filter_index_implied_dividend_yield(df_loaded)
df_filtered = df_filtered.sort_values(by=['date','expiration']).groupby('date').agg(list)
df_filtered[['expiration_near','expiration_next']] = pd.DataFrame(df_filtered['expiration'].tolist(), index=df_filtered.index)
df_filtered[['rate_near','rate_next']] = pd.DataFrame(df_filtered['rate'].tolist(), index=df_filtered.index)
df_filtered = df_filtered.drop(columns=['expiration','rate'])
df_filtered = df_filtered.filter(items=['date','expiration_near','expiration_next','rate_near','rate_next'])
df_filtered.index = [ts.date() for ts in df_filtered.index]


In [ ]:
df_filtered

In [ ]:
index_optionm_df["SPX"] = df_filtered

index_future_df["SPX"] = clean_bbg.get_clean_df(df_raw, date_ranges, index_pairs_map["SPX"]) 

index_df["SPX"] = df_raw["SPX" + " Index"]
index_df["SPX"].dropna(inplace=True)

In [ ]:
index_df['SPX'].head()

In [ ]:
index_future_df['SPX'].head()

In [ ]:
index_optionm_df['SPX'].head()

However, note that implied dividend yield is annualized thus, we need to take time to maturity into account

In [ ]:
index_optionm_df["SPX"][['days_to_near_expiry','days_to_far_expiry']] = index_optionm_df["SPX"][['expiration_near','expiration_next']].apply(lambda x: x - index_optionm_df["SPX"].index)
index_optionm_df["SPX"][['days_to_near_expiry','days_to_far_expiry']] = index_optionm_df["SPX"][['days_to_near_expiry','days_to_far_expiry']].applymap(lambda x: x.days)/360
index_optionm_df["SPX"][['div_near','div_next']] = index_optionm_df["SPX"][['rate_near','rate_next']] .apply(lambda x: x *index_df["SPX"]['PX_LAST'])/100
index_optionm_df["SPX"][['div_near', 'div_next']] = (
index_optionm_df["SPX"][['div_near', 'div_next']].values *
index_optionm_df["SPX"][['days_to_near_expiry', 'days_to_far_expiry']].values)

In [ ]:
index_optionm_df["SPX"]

STEP4 Annualizing and rescaling

Forward rate calculated with the formula I mentioned above is not annualized. We have to annualize it by a scale of inverse of remaining maturity
Likewise, we also calculate implied forward rates on OIS rate to compare it with forward rate embedded in futures.
Think it this way, if implied forward rate in futures > implied OIS 3M, you can enter short OIS3M ( pay fixed get float) and enter calendar spread. From calendar spread you will get fixed +alpha thus can earn float+alpha for t,tau1


In [ ]:
implied_forward_df = dict()
implied_forward_df["SPX"] = pd.DataFrame(index=index_future_df["SPX"].index, columns=['Implied Forward','Near Month TTM','Deferred Month TTM','Annualised'])
implied_forward_df["SPX"]['Implied Forward'] = (((index_future_df["SPX"]['Deferred Month PX_LAST']+index_optionm_df["SPX"]['div_next']) \
    /(index_future_df["SPX"]['Near Month PX_LAST']+index_optionm_df["SPX"]['div_near']))-1)*100
implied_forward_df["SPX"]['Near Month TTM'] = index_optionm_df["SPX"]['days_to_near_expiry']
implied_forward_df["SPX"]['Deferred Month TTM'] = index_optionm_df["SPX"]['days_to_far_expiry']
implied_forward_df["SPX"]['Annualised'] = (implied_forward_df["SPX"]['Implied Forward']) / (implied_forward_df["SPX"]['Deferred Month TTM'] - implied_forward_df["SPX"]['Near Month TTM']) 


In [ ]:

ois_df = df_raw['USSOC CMPN Curncy']['PX_LAST']
ois_df.dropna(inplace=True)

implied_forward_df['OIS'] = pd.DataFrame(index=ois_df.index, columns=['Annualised'])
implied_forward_df['OIS']['Annualised'] = ((1+ois_df/100 * implied_forward_df['SPX']['Deferred Month TTM'])/(1+ois_df/100 * implied_forward_df['SPX']['Near Month TTM'])-1)*4*100


In [ ]:
import matplotlib.pyplot as plt


total_df = pd.concat([implied_forward_df['SPX']['Annualised']

    ,implied_forward_df['OIS']],axis=1)
total_df.columns = ['SPX','OIS']
total_df.dropna(inplace=True)

total_df['SPX_Spread'] = total_df['SPX'] -total_df['OIS']
total_df['SPX_Spread'] *= 100



Final step. Plotting the spread


In [ ]:
import yfinance as yf



sp500 = yf.download('^GSPC', start=START_DATE, end=END_DATE)

fig, ax1 = plt.subplots(figsize=(12,6))

ax1.plot(total_df.index, total_df['SPX_Spread'],
         label='SPX Spread', linestyle='--', color='blue')
ax1.set_xlabel('Date')
ax1.set_ylabel('Spread (bps)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.hlines(0, total_df.index[0], total_df.index[-1],
           color='red', linestyles='--', linewidth=1)
ax1.set_ylim(-100, 150)

ax2 = ax1.twinx()
ax2.plot(sp500.index, np.log(sp500['Close']),
         label='SPX', linestyle='--', color='orange')
ax2.set_ylabel('SPX Index (log)', color='orange')
ax2.tick_params(axis='y', labelcolor='orange')


ax3 = ax1.twinx()

ax3.spines["right"].set_position(("axes", 1.2))
ax3.plot(ois_df.index, ois_df.values,  
         label='OIS', linestyle='--', color='green')
ax3.set_ylabel('OIS', color='green')
ax3.tick_params(axis='y', labelcolor='green')


plt.title('Equity Index Spread with OIS')
ax1.grid(True)


lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
lines_3, labels_3 = ax3.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2 + lines_3, labels_1 + labels_2 + labels_3, loc='best')

plt.show()
